## Setup

```
> conda create -n genslate python=3.6
> conda activate genslate
> conda install pip
> conda install ipykernel
> python -m ipykernel install --user --name genslate --display-name "GenSlate"
> conda install pytorch==1.7.0 torchvision==0.8.0 cudatoolkit=9.2 -c pytorch
> conda install -c conda-forge scikit-learn
> conda isntall tqdm setproctitle
```

### Datasets and Simulation Environment

See corresponding [notebook](Dataset%20and%20Simulation.ipynb).

## 1. Experiment

General process:
* Real-world data: pretrain response model on entire set --> train generative model on training set --> evaluate on test set
* Simulation: generate dataset from simulator --> pretrain response model on train+val set --> train generative model on train set --> evaluate on test set

Basic commands:

DATASET COMMAND examples:

* Yoochoose:
```
> --dataset yoochoose --nouser
> --dataset movielens
> --dataset movielens --nouser
```

* Simulation:
```
> --dataset urm --n_user 1000 --n_item 3000 --n_train 100000 --n_val 10000 --n_test 10000
> --dataset urmp --n_user 1000 --n_item 3000 --n_train 100000 --n_val 10000 --n_test 10000
> --dataset urmpmr --n_user 1000 --n_item 3000 --n_train 100000 --n_val 10000 --n_test 10000 --mr_factor 0.2
```

TRAINING COMMAND examples:

```
> --batch_size 64 --lr 0.001 --wdecay 0.001 --device cuda:0
> --batch_size 64 --lr 0.001 --wdecay 0.001 --device cuda:0 --nneg 100
```


### 1.1 Pretrain user response model from dataset

RESP MODEL COMMAND list:
* dim: embedding dimension size
* resp_struct: mlp structure as list of layer size

```
> python pretrain_env.py [DATASET COMMAND] [RESP MODEL COMMAND] [TRAINING COMMAND]
> python pretrain_env.py --dataset yoochoose --s 5 --nouser --dim 8 --resp_struct [40,256,256,5] --epoch 10 --batch_size 64 --lr 0.001 --wdecay 0.001 --device cuda:0
> python pretrain_env.py --dataset movielens --s 5 --dim 8 --resp_struct [48,256,256,5] --epoch 10 --batch_size 64 --lr 0.001 --wdecay 0.001 --device cuda:0
> python pretrain_env.py --dataset movielens --s 5 --nouser --dim 8 --resp_struct [40,256,256,5] --batch_size 64 --lr 0.001 --wdecay 0.001 --device cuda:0
> python pretrain_env.py --dataset urmp --sim_dim 8 --n_user 1000 --n_item 3000 --n_train 100000 --n_val 10000 --n_test 10000 --pbias_min=-0.2 --pbias_max 0.2 --dim 8 --resp_struct [48,256,256,5] --epoch 10 --batch_size 64 --lr 0.001 --wdecay 0.001 --device cuda:0
> python pretrain_env.py --dataset urmpmr --sim_dim 8 --n_user 1000 --n_item 3000 --n_train 100000 --n_val 10000 --n_test 10000 --pbias_min=-0.2 --pbias_max 0.2 --mr_factor 0.2 --dim 8 --resp_struct [48,256,256,5] --batch_size 64 --lr 0.001 --wdecay 0.001 --device cuda:0
```


### 1.2 Train generative model:

ENVIRONMENT COMMAND list:
* resp_path: saved response model path

MODEL COMMAND examples: (+8 for the first dim of each struct when --nouser flag is off)
```
> --model listcvae --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --dec_struct [22,256,256,40] --prior_struct [6,128,128] --mask_train
> --model pivotcvae --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --psm_struct [22,256,256,8] --scm_struct [30,256,256,32] --prior_struct [6,128,128] --mask_train
> --model pivotcvae_sgt_pi --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --psm_struct [22,256,256,8] --scm_struct [30,256,256,32] --prior_struct [6,128,128] --mask_train
> --model pivotcvae_sgt_pi --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --psm_struct [22,256,256,8] --scm_struct [30,256,256,32] --prior_struct [6,128,128] --mask_train
> --model pivotcvae_sgt_pi --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --psm_struct [22,256,256,8] --scm_struct [30,256,256,32] --prior_struct [6,128,128] --mask_train
```

Format:
```
> python train_generative.py [DATASET COMMAND] [ENVIRONMENT COMMAND] [MODEL COMMAND] [TRAINING COMMAND] --beta 0.001
```

Example 1: 
* Environment: yoochoose
* Model: listcvae
* Beta: 0.001

```
> python train_generative.py --dataset yoochoose --nouser --resp_path resp/yoochoose_nouser/resp_[40,256,256,5]_dim8_BS64_lr0.00100_decay0.00100 --model listcvae --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --dec_struct [22,256,256,40] --prior_struct [6,128,128] --mask_train --batch_size 64 --lr 0.0003 --wdecay 0.0001 --device cuda:0 --nneg 1000 --beta 0.001
```


Example 2: 
* Environment: movielens(User)
* Model: pivotcvae(SGT_SPI)
* Beta: 0.001

```
> python train_generative.py --dataset movielens --resp_path resp/movielens/resp_[48,256,256,5]_dim8_BS64_lr0.00100_decay0.00100 --model pivotcvae_sgt_spi --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --psm_struct [22,256,256,8] --scm_struct [30,256,256,32] --prior_struct [6,128,128] --mask_train --batch_size 64 --lr 0.0003 --wdecay 0.0001 --device cuda:0 --nneg 50 --beta 0.001
```


Example 3: 
* Environment: URM_P_MR
* Model: pivotcvae
* Beta: entire search space in settings.py

```
> python train_generative.py --dataset urmpmr --n_user 1000 --n_item 3000 --n_train 100000 --n_val 10000 --n_test 10000 --mr_factor 0.2 --resp_path resp/urmpmr_dim8_u1000_i3000_n100000_mr0.20/resp_[48,256,256,5]_dim8_BS64_lr0.00100_decay0.00100 --model pivotcvae --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --psm_struct [22,256,256,8] --scm_struct [30,256,256,32] --prior_struct [6,128,128] --mask_train --batch_size 64 --lr 0.0003 --wdecay 0.0001 --device cuda:0 --nneg 50 --beta -1
```
